In [1]:
# Import modules
import logging
#!pip install wikipedia-api
import wikipediaapi
import databasesaver as dbs
import requests
import pandas as pd
import os
import re

notebook_path = os.getcwd()
print(notebook_path)

/Anton1G3/Study/ds23_f-rdjupad_python/lektion_5


# Explode styles and insribers

In [42]:
df = pd.read_csv('Uppland.csv')
#df.loc[1080]
# convert to w2v+explode
# X, y train -> predict
#df["Inscriber"] = df["Inscriber"].fillna(0)
df["Inscriber"] = df["Inscriber"].str.split(r"\s*,\s*", regex=True)
x = df.explode("Inscriber")
df["Stil"] = df["Stil"].str.split(r"\s*,\s*", regex=True)
y = df.explode("Stil")
df_out = (
    pd.concat(
             [df.set_index("signum"), pd.crosstab(x["signum"], x["Inscriber"]), pd.crosstab(y["signum"], y["Stil"])], axis=1
    )
    .reset_index()
    .drop(columns=["Inscriber","Stil"])
)

#result = df[df['Period_end'] - df['Period_begin'] < 31]


#mask = df['Inscriber'].str.contains(r'Tidkume', na=False) #Öpir
#result = df[mask]

df_out = df_out.fillna(0)
df_out

,signum,revisionID,translitterering,normalisering,translation,Latitude,Longitude,Material,Period_begin,Period_end,...,Östen 2,þuliR,Fp,KB,RAK,pr1,pr2,pr3,pr4,pr5
0,Brostenen,54953925,kinluk × hulmkis × tutiʀ × systiʀ × sukruþaʀ ×...,"Ginnlaug, Holmgæiʀs dottiʀ, systiʀ Sygrøðaʀ ok...","Ginnlög, Holmgers dotter, syster till Sygröd o...",59.50824,17.62721,runsten,793,1066,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,Bökstastenen,51724326,· inki-... [a]uk : iuker [:] þau : litu : rais...,Ingi... ok Iogærðr þau letu ræisa stæin þenna ...,Inge-... och Jogard de läto resa denna sten ef...,59.76891,17.41231,runsten,1000,1066,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,Folsbergastenen,52768827,+ akyti lit + rita + st-n + þino + yfti + tumo...,Aguti let retta st[æi]n þenna æftiR Tuma/Tumma...,Agute lät resa denna sten efter Tumme [sin fad...,59.56056,17.17538,0,793,1066,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Gliastenen,55417455,× k[a]mal ak × þialfi × þiʀ × litu × risa × s(...,Gamall ok Þialfi þæiʀ letu ræisa stæin þennsa ...,Gammal och Tjälve de läto resa denna sten (eft...,59.34638,17.95199,Runsten,793,1066,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,"Granbyhällen, Husby",55339835,ikilauh × lit + hakua · eli þsa · eftʀ + sun s...,Ingilaug let haggva hælli þessa æftiʀ sun sinn...,Ingelög lät hugga denna häll efter sin son Sig...,59.40457,17.92723,runhäll,1000,1099,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,Södermanlands runinskrifter Sb1965;12,55455260,: ormr : auk : faskr : auk : þora :· litu : ra...,Ormʀ ok Fastgæiʀʀ/Fastgærðr/Fauskʀ ok Þora let...,Orm och Fastgeirr/Fastgerðr/Fauskr och Tora lä...,59.42573,16.33903,0,793,1066,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1409,Södermanlands runinskrifter SB1965;19,20074180,ku...---- ...--... kuna + l-...u + hakua + sta...,... ... Gunna le[t]u haggva stæin þenna ok bro...,Gu...(och) Gunna läto hugga denna sten och (gö...,0.00000,0.00000,0,0,1500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1410,Runmarsvreten,55883165,ku...---- ...--... kuna + l-...u + hakua + sta...,... ... Gunna le[t]u haggva stæin þenna ok bro...,Gu...(och) Gunna läto hugga denna sten och (gö...,59.15045,18.17988,runhäll,793,1066,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1411,Södermanlands runinskrifter SB1965;20,20074175,-(u)--... -u(k) ... ...--t----(a)...- (f)(þ)(u...,... ok() ... ... faður() Guð hialpi and han[s]...,...och()...fader() Gud hjälpe hans ande.,0.00000,0.00000,0,0,1500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# cut runic text i words
#! pip install text-preprocessing
from text_preprocessing import preprocess_text
df_out['translitterering'] = df_out['translitterering'].str.replace('(','')
df_out['translitterering'] = df_out['translitterering'].str.replace(')','')
df_out['translitterering'] = df_out['translitterering'].str.replace('[','')
df_out['translitterering'] = df_out['translitterering'].str.replace(']','')
df_out['translitterering'] = df_out['translitterering'].str.replace('...','')
df_out['translitterering'] = df_out['translitterering'].str.replace(r"\s*×\s*",' ', regex=True)
df_out['translitterering'] = df_out['translitterering'].str.replace(r"\s*\+\s*",' ', regex=True)
df_out['translitterering'] = df_out['translitterering'].str.replace(r"\s*:\s*",' ', regex=True)
df_out['translitterering'] = df_out['translitterering'].str.replace(r"\s*\|\s*",' ', regex=True)
df_out['translitterering'] = df_out['translitterering'].str.replace(r"\s*·\s*",' ', regex=True)
#df_out['translitterering'] = df_out['translitterering'].str.replace(r"\s*-\s*",'', regex=True)
df_out['translitterering'] = df_out['translitterering'].str.replace(r"\s*\.\s*",' ', regex=True)
#df_out['translitterering'] = df_out['translitterering'].str.replace(r"\s*,\s*",' ', regex=True)
df_out['translitterering'] = df_out['translitterering'].str.split()

[nltk_data] Downloading package omw-1.4 to /Users/angr/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
# get Word2Vec model
#! pip install gensim
from gensim.models import Word2Vec
Word2Vec_model = Word2Vec(df_out['translitterering'], seed=0, workers=4, sg=0)

In [36]:
# one can really have fun with Word2Vec
print(Word2Vec_model.wv.most_similar(positive=['stain'], topn=10))
Word2Vec_model.wv['stain']


[('auk', 0.9995585083961487), ('at', 0.9994363784790039), ('sin', 0.9993960857391357), ('stin', 0.9993941187858582), ('bru', 0.9993917346000671), ('stan', 0.9993906617164612), ('litu', 0.9993825554847717), ('a', 0.9993726015090942), ('sun', 0.9993721842765808), ('til', 0.9993554949760437)]


array([ 0.04138811, -0.21391557, -0.02111843, -0.3143679 ,  0.2787874 ,
        0.07077538, -0.12365589, -0.48742977,  0.10130572,  0.1880094 ,
        0.2405688 ,  0.10236811,  0.25534266,  0.4280772 ,  0.31930357,
       -0.0089304 ,  0.02272191, -0.3031695 , -0.31360686,  0.20200346,
        0.0031526 ,  0.37713996, -0.03469337, -0.3363603 ,  0.25294045,
        0.23694251,  0.11561681,  0.13001245, -0.01674494, -0.10666823,
        0.17235565, -0.08809467,  0.14213423, -0.00658507, -0.43731752,
       -0.04672825,  0.09348158, -0.17608614, -0.00113934,  0.26653934,
       -0.03452369, -0.3974576 , -0.12508468, -0.06816301, -0.21105656,
       -0.18912204, -0.2474511 ,  0.04166899, -0.16986448,  0.17722233,
       -0.24612933,  0.0591435 , -0.20149206,  0.60305077,  0.03879384,
       -0.36921036, -0.10691167,  0.18350992, -0.11931608,  0.21193273,
       -0.02150366,  0.39401188, -0.06437674, -0.3128894 ,  0.6233673 ,
       -0.04372983, -0.21970865,  0.12050993,  0.15459123,  0.11

In [35]:
Word2Vec_model = Word2Vec.load("word2vec.model")
print(Word2Vec_model.wv.most_similar(positive=['stain'], topn=10))
Word2Vec_model.wv['stain']


[('auk', 0.9995585083961487), ('at', 0.9994363784790039), ('sin', 0.9993960857391357), ('stin', 0.9993941187858582), ('bru', 0.9993917346000671), ('stan', 0.9993906617164612), ('litu', 0.9993825554847717), ('a', 0.9993726015090942), ('sun', 0.9993721842765808), ('til', 0.9993554949760437)]


array([ 0.04138811, -0.21391557, -0.02111843, -0.3143679 ,  0.2787874 ,
        0.07077538, -0.12365589, -0.48742977,  0.10130572,  0.1880094 ,
        0.2405688 ,  0.10236811,  0.25534266,  0.4280772 ,  0.31930357,
       -0.0089304 ,  0.02272191, -0.3031695 , -0.31360686,  0.20200346,
        0.0031526 ,  0.37713996, -0.03469337, -0.3363603 ,  0.25294045,
        0.23694251,  0.11561681,  0.13001245, -0.01674494, -0.10666823,
        0.17235565, -0.08809467,  0.14213423, -0.00658507, -0.43731752,
       -0.04672825,  0.09348158, -0.17608614, -0.00113934,  0.26653934,
       -0.03452369, -0.3974576 , -0.12508468, -0.06816301, -0.21105656,
       -0.18912204, -0.2474511 ,  0.04166899, -0.16986448,  0.17722233,
       -0.24612933,  0.0591435 , -0.20149206,  0.60305077,  0.03879384,
       -0.36921036, -0.10691167,  0.18350992, -0.11931608,  0.21193273,
       -0.02150366,  0.39401188, -0.06437674, -0.3128894 ,  0.6233673 ,
       -0.04372983, -0.21970865,  0.12050993,  0.15459123,  0.11

In [37]:

# we have vectorzed words, now we need to vectorize sentences

import numpy as np
# Creating empty dataframe to hold sentences
W2Vec_data=pd.DataFrame()
    
# Looping through each row for the data
#count = 0
for sentence in df_out['translitterering']:
    #count = count + 1
    #if count > 4 : break
    #print(sentence)
# initiating a sentence with all zeros
    vec_sentence = np.zeros(100)

# Looping thru each word in the sentence and if its present in 
# the Word2Vec model then storing its vector
    for word in sentence:
        #print(word)
        if word in Word2Vec_model.wv.key_to_index.keys() :
            vec_sentence=vec_sentence+Word2Vec_model.wv[word]
        # Appending the sentence to the dataframe
    W2Vec_data=pd.concat([W2Vec_data,pd.DataFrame([vec_sentence])],ignore_index=True)
#return(W2Vec_Data)

In [44]:
W2Vec_data = W2Vec_data*6

In [45]:
#create train data

df_ML = pd.concat([W2Vec_data, df_out], axis=1)
df_ML = df_ML.drop(columns=["signum","revisionID","translitterering","normalisering","translation","Material","edition"])
X = df_ML[df_ML['Period_end'] - df_ML['Period_begin'] < 31]
y = (np.array(X['Period_end']) + np.array(X['Period_begin']))/2


begin_period = [1043 - 8.5, (1097 + 1108)/2 - 5, 1085 - 14, 1050 - 5, 1066 - 9.5, 1085 - 5]
end_period = [1043 + 8.5, (1097 + 1108)/2 + 5, 1085 + 14, 1050 + 5, 1066 + 9.5, 1085 + 5]

count = 0
for insk in ["Upplands runinskrifter 35", "Upplands runinskrifter 209", "Orkestastenen", "Upplands runinskrifter 644", "Upplands runinskrifter 11", "Upplands runinskrifter 229"] :
    X = pd.concat([X,df_ML.loc[df_out['signum'] == insk]],ignore_index=True)
    X.at[len(X)-1, 'Period_begin'] = begin_period[count]
    X.at[len(X)-1, 'Period_end'] = end_period[count]
    y = np.append( y, (begin_period[count] + end_period[count])/2 )
    count = count + 1
        #if df.loc[df['signum'] == inskrift, 'revisionID'].item() < page_data['parse']['revid']:
X = X.fillna(0)


In [23]:
# train the model
from sklearn.neighbors import KNeighborsRegressor
X.columns = X.columns.astype(str)
neigh = KNeighborsRegressor(n_neighbors=5)
neigh.fit(X, y)


KNeighborsRegressor()

In [33]:
import pickle 
     
# load the model from disk
neigh = pickle.load(open('KNeighborsRegressor_pickle_file', 'rb'))
#result = neigh.predict(X_test) 

In [48]:
# test on iscribers like Tidkume (person, see pdf) and Öpir (verksamhet)
count = 0
mask = df['Inscriber'].str.contains('Tidkume', na=False, regex=False) #Öpir Tidkume
#print(df[mask])

#X_pred = df_ML["Öpir" in df["Inscriber"]]
X_pred = df_ML[mask]
X_pred = X_pred.fillna(0)
X_pred.columns = X_pred.columns.astype(str)
print(neigh.predict(X_pred))
#coordinates are available in 
df[mask]


[1048.6 1023.5 1023.5 1023.5 1019.  1022.  1023.5 1023.5 1029.5 1014.5
 1026.5 1023.5 1019.  1023.5 1029.5]


,signum,revisionID,translitterering,normalisering,translation,Latitude,Longitude,Stil,Inscriber,Material,Period_begin,Period_end,edition
606,Upplands runinskrifter 691,54589581,birn : lit : risa : s--- ...- : iftiʀ : sterbi...,Biorn let ræisa s[tæin þennsa] æftiʀ Styrbiorn...,"Björn lät resa (denna sten) efter Styrbjörn, s...",59.49639,17.12510,NaN,"[Ödbjörn enligt Erik Brate, Brate, ej troligt...",NaN,1035,1065,0
622,Upplands runinskrifter 718,54593741,[...asr : auk : olifʀ : auk : uar:asi : li-......,... ok Olæifʀ ok Var-Asi()/Varr-Asi() le[tu] ....,... och Olev och '''uarasi''' läto ... denna s...,59.55498,17.14018,NaN,"[Tidkume, osäker bedömning]",NaN,793,1066,0
635,Upplands runinskrifter 733,54537884,[(þ)(i)(l)stan : lit : ris... ... ...n : sun :...,"Þorstæinn() let ræis[a] ... ..., sun sinn goða...","Torsten() lät resa ..., sin gode son. Tidkume ...",59.59654,17.30669,NaN,[Tidkume],NaN,793,1066,0
644,Upplands runinskrifter 742,54631331,+ ari + lit × ʀftia × usyrk · faþur + sin + --...,"Ari let æftiʀ Osyrg, faður sinn. [Tið]kumi ris...","Are lät (resa stenen) efter Osyrg, sin fader ....",59.60033,17.12405,NaN,"[Tidkume, Livsten]",NaN,793,1066,0
657,Upplands runinskrifter 758,51521048,ikib(i)rn : ak : þiʀ : bryþr : litu : risa : s...,Ingibiorn ok þæiʀ brøðr letu ræisa stæin þenna...,Ingebjörn och bröderna med honom läto resa den...,59.63749,17.08005,[pr4],[Tidkume],NaN,793,1066,0
658,Upplands runinskrifter 759,51514457,birynhiifʀ : ak : hrifnkʀ : ak : sik:ualti : u...,Bryniulfʀ() ok Hræfningʀ() ok Sigvaldi letu ræ...,Brynjulv och Rävning () och Sigvalde läto resa...,59.63898,17.08096,[pr4],[Tidkume],NaN,793,1066,0
661,Upplands runinskrifter 763,54686058,bruni : ak : hulm:stin : litu : risa : istin :...,Bruni ok Holmstæinn letu ræisa stæin þennsa æf...,Brune och Holmsten läto resa stenen efter Gunn...,59.61380,17.10020,NaN,[Samma ristare som gjort Upplands runinskrifte...,NaN,793,1066,0
662,Upplands runinskrifter 764,54688368,brunk : ak : hulm:stin : litu : risa : istin :...,Bruni ok Holmstæinn letu ræisa stæin þennsa æf...,Brune och Holmsten läto resa denna sten efter ...,59.61386,17.10017,NaN,"[Samma som gjort Upplands runinskrifter 763, U...",NaN,793,1066,0
663,Upplands runinskrifter 765,54688642,-...k...-...-n : h...sl-...- : a-s : -...,... ... ... ...,NaN,59.61276,17.09235,NaN,[Tidkume],NaN,793,1066,0
671,Upplands runinskrifter 775,54599487,u[i]tarfʀ · lit · rsa · stin · linsa · ifi[ʀ ·...,Vidiarfʀ let ræisa stæin þennsa æftiʀ Asgæiʀ()...,Vidjärv lät resa denna sten efter Åsger () och...,59.56718,17.02141,NaN,[Tidkume],NaN,793,1066,0


In [47]:
# tegn stenar från Vg correspond to Olof Skötkonung late ruling dates https://sv.wikipedia.org/wiki/Olof_Sk%C3%B6tkonung
mask = df['translitterering'].str.contains('þikn', na=False, regex=False) #Öpir Tidkume þegn þikn
#print(df[mask])
X_pred = df_ML[mask]
X_pred = X_pred.fillna(0)
X_pred.columns = X_pred.columns.astype(str)
print(neigh.predict(X_pred)) 

[1025.  1023.5 1023.5 1026.5 1014.5 1014.5 1019.  1014.5 1014.5 1014.5]


In [46]:
# Ingvars sten is right after Ingvars tåget
mask = df['translitterering'].str.contains('ikuars', na=False, regex=False) #Öpir Tidkume þegn þikn
#print(df[mask])

X_pred = df_ML[mask]
X_pred = X_pred.fillna(0)
X_pred.columns = X_pred.columns.astype(str)
print(neigh.predict(X_pred)) 

[1042.9]


In [40]:
df[mask]
X_pred

,0,1,2,3,4,5,6,7,8,9,...,Östen 2,þuliR,Fp,KB,RAK,pr1,pr2,pr3,pr4,pr5
674,0.837201,-3.649044,-0.424835,-5.322698,4.738164,1.124167,-2.074769,-8.489754,1.918841,3.168263,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
